In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 1. Daten Lesen
df = pd.read_csv("data/tesla_top5_daily.csv", parse_dates=["created"])
embeddings = np.load("data/embedding_data/embeddings_complete_scaled.npy")

assert len(df) == len(embeddings), "Anzahl Zeilen stimmt nicht mit Embeddings überein!"

# 2. Timestamp in Wochenzeitraum überführen (Freitag 16 Uhr bis Freitag 16 Uhr)

def get_week_bucket(dt):
    # Setze auf Freitag 16 Uhr
    dt = pd.Timestamp(dt)
    weekday = dt.weekday()
    hour = dt.hour

    # Freitag = 4; wenn nach Freitag 16 Uhr, gehört es zur nächsten Woche
    if (weekday > 4) or (weekday == 4 and hour >= 16):
        # nächster Freitag
        days_until_next_friday = (11 - weekday) % 7
        friday = dt + timedelta(days=days_until_next_friday)
    else:
        # letzte Freitag
        days_since_last_friday = (weekday - 4) % 7
        friday = dt - timedelta(days=days_since_last_friday)

    # Setze Uhrzeit auf Freitag 16:00:00
    bucket = pd.Timestamp(friday.date()) + pd.Timedelta(hours=16)
    return bucket

df["week"] = df["created"].apply(get_week_bucket)

# 3. Gruppieren
grouped_embeddings = {}
for week, group in df.groupby("week"):
    indices = group.index.tolist()
    group_emb = embeddings[indices]
    grouped_embeddings[week.strftime("%Y-%m-%d")] = group_emb

# 4. Speichern
np.save("data/embedding_data/weekly_embeddings_dict.npy", grouped_embeddings)

print(f"{len(grouped_embeddings)} Wochen gebildet.")


552 Wochen gebildet.
